# LibriSpeech Dataset

In [ ]:
!wget -q https://www.openslr.org/resources/12/dev-clean.tar.gz
!tar -xvzf dev-clean.tar.gz --wildcards --no-anchored "*.flac" -C /content/
audio_file_path = "/content/LibriSpeech/dev-clean/1272/128104/1272-128104-0000.flac"

LibriSpeech/dev-clean/2277/149896/2277-149896-0026.flac
LibriSpeech/dev-clean/2277/149896/2277-149896-0005.flac
LibriSpeech/dev-clean/2277/149896/2277-149896-0033.flac
LibriSpeech/dev-clean/2277/149896/2277-149896-0006.flac
LibriSpeech/dev-clean/2277/149896/2277-149896-0018.flac
LibriSpeech/dev-clean/2277/149896/2277-149896-0034.flac
LibriSpeech/dev-clean/2277/149896/2277-149896-0021.flac
LibriSpeech/dev-clean/2277/149896/2277-149896-0015.flac
LibriSpeech/dev-clean/2277/149896/2277-149896-0012.flac
LibriSpeech/dev-clean/2277/149896/2277-149896-0027.flac
LibriSpeech/dev-clean/2277/149896/2277-149896-0007.flac
LibriSpeech/dev-clean/2277/149896/2277-149896-0030.flac
LibriSpeech/dev-clean/2277/149896/2277-149896-0011.flac
LibriSpeech/dev-clean/2277/149896/2277-149896-0009.flac
LibriSpeech/dev-clean/2277/149896/2277-149896-0003.flac
LibriSpeech/dev-clean/2277/149896/2277-149896-0004.flac
LibriSpeech/dev-clean/2277/149896/2277-149896-0017.flac
LibriSpeech/dev-clean/2277/149896/2277-149896-00

# Learn Audio Shape

In [ ]:
import torchaudio

# Replace 'path_to_audio_file.wav' with the path to your downloaded file
signal, sample_rate = torchaudio.load('/content/LibriSpeech/dev-clean/422/122949/422-122949-0000.flac')
print(f"Shape of audio signal: {signal.shape}")  # Expected: [channels, sequence_length]
print(f"Sample rate: {sample_rate}")

Shape of audio signal: torch.Size([1, 323520])
Sample rate: 16000


# Convert Audio Shape to 16Hz

In [ ]:
!pip install torchaudio scipy


import torchaudio
import torch
from scipy.io.wavfile import write
import os


def download_sample_audio():
    url = "https://www2.cs.uic.edu/~i101/SoundFiles/preamble10.wav"
    audio_path = "sample_audio.wav"
    if not os.path.exists(audio_path):
        !wget -O {audio_path} {url}
    return audio_path


def convert_audio(input_path, output_path, desired_duration=5.0, desired_sample_rate=16000):

    if not os.path.exists(input_path):
        raise FileNotFoundError(f"The specified audio file does not exist: {input_path}")

    signal, sample_rate = torchaudio.load(input_path)

    if sample_rate != desired_sample_rate:
        resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=desired_sample_rate)
        signal = resampler(signal)
        sample_rate = desired_sample_rate

    if signal.shape[0] > 1:
        signal = signal.mean(dim=0, keepdim=True)

    num_samples = int(desired_sample_rate * desired_duration)
    if signal.shape[1] > num_samples:
        signal = signal[:, :num_samples]
    else:
        padding = num_samples - signal.shape[1]
        signal = torch.nn.functional.pad(signal, (0, padding))

    signal = signal.squeeze().unsqueeze(0)

    signal_np = signal.numpy()
    write(output_path, sample_rate, signal_np[0])

    print(f"Audio saved to: {output_path}")
    return output_path


input_audio_path = download_sample_audio()
output_audio_path = "converted_audio.wav"
converted_audio_path = convert_audio(input_audio_path, output_audio_path, desired_duration=5.0, desired_sample_rate=16000)


--2024-11-08 15:37:41--  https://www2.cs.uic.edu/~i101/SoundFiles/preamble10.wav
Resolving www2.cs.uic.edu (www2.cs.uic.edu)... 131.193.32.16
Connecting to www2.cs.uic.edu (www2.cs.uic.edu)|131.193.32.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 423364 (413K) [audio/x-wav]
Saving to: ‘sample_audio.wav’

sample_audio.wav    100%[===================>] 413.44K  --.-KB/s    in 0.1s    

2024-11-08 15:37:41 (3.48 MB/s) - ‘sample_audio.wav’ saved [423364/423364]

Audio saved to: converted_audio.wav


# Speech to Emotion Detection

In [ ]:
!pip install transformers torchaudio gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7


In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, pipeline
import torch
import torchaudio
import gradio as gr

In [ ]:
asr_processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
asr_model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

In [ ]:
emotion_recognition = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
def predict_emotion_from_audio(audio):
    try:
        # Load the audio file
        signal, fs = torchaudio.load(audio)

        # Resample to 16kHz if necessary for ASR model compatibility
        if fs != 16000:
            resampler = torchaudio.transforms.Resample(orig_freq=fs, new_freq=16000)
            signal = resampler(signal)

        # Convert stereo audio to mono
        if signal.shape[0] > 1:
            signal = signal.mean(dim=0)

        # Ensure the shape of signal is [1, sequence_length] for ASR model compatibility
        signal = signal.unsqueeze(0) if signal.ndim == 1 else signal  # If it's 1D, make it 2D
        signal = signal.squeeze(0) if signal.shape[0] == 1 else signal  # Remove extra batch dimension if needed

        # Process the audio with the processor to prepare it for the model
        input_values = asr_processor(signal, sampling_rate=16000, return_tensors="pt").input_values.squeeze()

        # Predict the transcription
        with torch.no_grad():
            logits = asr_model(input_values.unsqueeze(0)).logits  # Ensure input to model is [1, sequence_length]
            predicted_ids = torch.argmax(logits, dim=-1)
            transcription = asr_processor.decode(predicted_ids[0])

        # Run emotion recognition on transcribed text
        emotions = emotion_recognition(transcription)

        # Format the output to show each emotion score
        output = "Transcription: " + transcription + "\n\nEmotion Scores:\n"
        for emotion in emotions[0]:
            output += f"{emotion['label']}: {emotion['score']:.2f}\n"

        return output

    except Exception as e:
        return f"Error processing audio: {e}"

In [ ]:
iface = gr.Interface(
    fn=predict_emotion_from_audio,
    inputs=gr.Audio(type="filepath"),
    outputs="text",
    title="Emotion Detection from Speech",
    description="Upload an audio file to detect the emotion expressed in the speech."
)

In [ ]:
iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c26c9e3acd20d5c48e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
